In [7]:
pip install transformers torch --break-system-packages --quiet && echo "Dependencies installed successfully"

Dependencies installed successfully


In [8]:
"""
Environment Check Script: gtfintechlab/FOMC-RoBERTa Model Validation
Purpose: Sanity check for gated model access, dynamic label mapping, and OOD pipeline skeleton.
Usage: Run this before proceeding to notebooks to verify model availability and inference logic.
Author: Shanhuizi (Mia) Jiang
"""

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Global variable to track which model is actually being used
ACTIVE_MODEL_NAME = None

def load_model_and_config():
    """Load model and dynamically retrieve label mappings from model config"""
    global ACTIVE_MODEL_NAME

    print("=" * 60)
    print("STEP 1: Loading Model and Tokenizer")
    print("=" * 60)

    primary_model = "gtfintechlab/FOMC-RoBERTa"
    fallback_model = "ProsusAI/finbert"

    # Try primary model first
    try:
        print(f"Trying primary model: {primary_model}")
        tokenizer = AutoTokenizer.from_pretrained(primary_model)
        model = AutoModelForSequenceClassification.from_pretrained(primary_model)

        ACTIVE_MODEL_NAME = primary_model

        print(f"✓ Successfully loaded primary model: {primary_model}")
        print(f"✓ Model has {model.config.num_labels} labels")
        print(f"✓ Label mapping (id2label): {model.config.id2label}")

        return tokenizer, model

    except Exception as e:
        print(f"✗ Failed to load primary model ({primary_model}): {e}")
        print("\n⚠ Detected gated / restricted model.")
        print("  → For FSIL internal environment: provide a Hugging Face token with access to this repo.")
        print("    e.g., set HF_TOKEN in environment or run `huggingface-cli login` on lab machines.")
        print("  → For external demo: falling back to a public financial model for pipeline development.\n")

    # Fallback to public model
    try:
        print(f"Trying fallback model: {fallback_model}")
        tokenizer = AutoTokenizer.from_pretrained(fallback_model)
        model = AutoModelForSequenceClassification.from_pretrained(fallback_model)

        ACTIVE_MODEL_NAME = fallback_model

        print(f"✓ Successfully loaded fallback model: {fallback_model}")
        print(f"✓ Model has {model.config.num_labels} labels")
        print(f"✓ Label mapping (id2label): {model.config.id2label}")

        return tokenizer, model

    except Exception as e:
        print(f"✗ Failed to load fallback model: {e}")
        return None, None


def test_inference(tokenizer, model):
    """Run inference on diverse financial narratives (2024-2025 OOD scenarios)"""
    print("\n" + "=" * 60)
    print("STEP 2: Running Inference on OOD Sample Texts")
    print("=" * 60)

    # Sanity checks on model configuration
    assert model.config.num_labels >= 2, "Model must have at least 2 labels for sentiment classification."
    assert isinstance(model.config.id2label, (dict, list)), "id2label must be dict or list-like structure."

    print(f"🔹 Active model: {ACTIVE_MODEL_NAME}")
    print(f"🔹 Label semantics: {model.config.id2label}")
    print()
    print("📝 Context for Test Cases:")
    print("   These test cases are written in FOMC-style narratives (2024-2025 OOD).")

    if "finbert" in ACTIVE_MODEL_NAME.lower():
        print("   → Current model (FinBERT) uses generic financial sentiment:")
        print("      positive / negative / neutral")
        print("   → In FSIL internal runs with FOMC-RoBERTa, the same pipeline will operate on:")
        print("      dovish / neutral / hawkish labels")
        print("   → This script validates the pipeline logic, not final label interpretation.\n")
    else:
        print("   → Using FOMC-RoBERTa with Fed-specific sentiment labels.\n")

    # Intentionally selected 2024-2025 narrative styles for OOD testing

    # TODO (FSIL internal runs):
    # Replace these handcrafted sentences with real 2024-2025 FOMC excerpts
    # once data_collection_and_cleaning.ipynb is ready.
    test_cases = [
        {
            "text": "The Committee decided to maintain the target range for the federal funds rate at 5.25 to 5.5 percent.",
            "expected": "Neutral/Hawkish (maintaining high rates)"
        },
        {
            "text": "Inflation has eased over the past year but remains elevated. The Committee remains highly attentive to inflation risks.",
            "expected": "Hawkish (emphasizing inflation concerns)"
        },
        {
            "text": "The Committee decided to lower the target range for the federal funds rate by 1/2 percentage point to 4.75 to 5 percent.",
            "expected": "Dovish (rate cut - Sept 2024 scenario)"
        },
        {
            "text": "Economic activity has been expanding at a solid pace. The labor market remains strong.",
            "expected": "Neutral/Hawkish (strong economy)"
        },
        {
            "text": "The Committee will continue to assess additional information and its implications for monetary policy. We are prepared to be patient.",
            "expected": "Neutral? (But market may read 'patient' as Hawkish delay)"
        }
    ]


    id2label = model.config.id2label

    def get_label(label_id: int):
        """
        Safely retrieve label name, compatible with:
        - dict with string keys: {'0': 'Dovish', '1': 'Neutral', '2': 'Hawkish'}
        - dict with int keys: {0: 'Dovish', 1: 'Neutral', 2: 'Hawkish'}
        - list: ['Dovish', 'Neutral', 'Hawkish']
        """
        if isinstance(id2label, dict):
            # Try both string and int keys
            return id2label.get(str(label_id), id2label.get(label_id, f"label_{label_id}"))
        else:
            # Assume list-like
            return id2label[label_id]

    model.eval()

    for i, case in enumerate(test_cases, 1):
        print(f"\n--- Test Case {i} ---")
        print(f"Text: \"{case['text'][:100]}{'...' if len(case['text']) > 100 else ''}\"")
        print(f"Expected Sentiment (by FOMC intuition): {case['expected']}")

        inputs = tokenizer(
            case['text'],
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        )

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
            predicted_class = torch.argmax(probs, dim=-1).item()

        print(f"→ Model Prediction: [{get_label(predicted_class)}] (confidence: {probs[0][predicted_class].item():.3f})")
        print(f"  Confidence Distribution:")
        for label_id in range(model.config.num_labels):
            print(f"    {get_label(label_id)}: {probs[0][label_id].item():.4f}")

        # Additional context for FOMC-RoBERTa runs
        if "fomc" in ACTIVE_MODEL_NAME.lower():
            print(f"  (Sanity note: FOMC expectation is '{case['expected']}')")


def main():
    print("\n" + "=" * 60)
    print("🚀 ENVIRONMENT CHECK: FOMC-RoBERTa OOD Stress Test")
    print("   Step 0: Validate Model Access & Pipeline Skeleton")
    print("=" * 60 + "\n")

    tokenizer, model = load_model_and_config()

    if model is None:
        print("\n✗ ENVIRONMENT CHECK FAILED: Cannot proceed without model")
        print("   Please resolve model access issues before running notebooks.")
        return

    test_inference(tokenizer, model)

    print("\n" + "=" * 60)
    print("✓ ENVIRONMENT READY FOR FSIL DEMO PROJECT")
    print("=" * 60)
    print("\n📋 Next Steps:")

    print("  1. Model environment validated ✓")
    print("  2. Proceed to notebooks/1_data_collection_2024_2025.ipynb")
    print("     → Ingest real 2024-2025 FOMC statements")
    print("  3. Continue to notebooks/2_ood_generalization_cases.ipynb")
    print("     → Systematic OOD failure case analysis with market context")
    print("  4. Finalize with notebooks/3_event_study_backtest.ipynb")
    print("     → Connect to market data (SPY, yields) for Sharpe/Drawdown metrics")

    print("\n💡 Design Note:")
    print("   This script respects the original model's label design.")
    print("   No hardcoded assumptions - ready for any label configuration.")
    print("\n👉 Usage Summary:")
    print("   • Verify gated access to gtfintechlab/FOMC-RoBERTa, or")
    print("   • Prototype the full OOD evaluation pipeline with public FinBERT.")
    print("   • Both modes test identical pipeline logic with different label spaces.")


if __name__ == "__main__":
    main()


🚀 ENVIRONMENT CHECK: FOMC-RoBERTa OOD Stress Test
   Step 0: Validate Model Access & Pipeline Skeleton

STEP 1: Loading Model and Tokenizer
Trying primary model: gtfintechlab/FOMC-RoBERTa
✗ Failed to load primary model (gtfintechlab/FOMC-RoBERTa): You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/gtfintechlab/FOMC-RoBERTa.
401 Client Error. (Request ID: Root=1-69369a31-785bc7c5524a5403376b0eb4;7250f0d3-9463-4571-a31e-6398d82b74c4)

Cannot access gated repo for url https://huggingface.co/gtfintechlab/FOMC-RoBERTa/resolve/main/config.json.
Access to model gtfintechlab/FOMC-RoBERTa is restricted. You must have access to it and be authenticated to access it. Please log in.

⚠ Detected gated / restricted model.
  → For FSIL internal environment: provide a Hugging Face token with access to this repo.
    e.g., set HF_TOKEN in environment or run `huggingface-cli login` on lab machines.
  → For external demo: falling back to a public financial mod